In [1]:
 import pandas as pd
    

In [2]:
df = pd.read_csv("./buildings.csv")
df.head()

,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
1,Shanghai Tower,Shanghai,China,632,123,2015
2,Abraj Al-Bait Clock Tower,Mecca,Saudi Arabia,601,120,2012
3,Ping An Finance Center,Shenzhen,China,599,115,2015
4,One World Trade Center,New York City,United States,541,104,2014


In [3]:
df.describe()

,Height,Floors,Built
count,121.000000,121.000000,121.000000
mean,358.165289,77.264463,2006.735537
std,79.653034,17.799236,13.691097
min,300.000000,36.000000,1930.000000
25%,308.000000,68.000000,2007.000000
50%,329.000000,75.000000,2011.000000
75%,380.000000,85.000000,2015.000000
max,828.000000,163.000000,2015.000000


We want to get the first 10 results   
`SELECT * FROM buildings LIMIT 10`


In [4]:
df[:10]

,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
1,Shanghai Tower,Shanghai,China,632,123,2015
2,Abraj Al-Bait Clock Tower,Mecca,Saudi Arabia,601,120,2012
3,Ping An Finance Center,Shenzhen,China,599,115,2015
4,One World Trade Center,New York City,United States,541,104,2014
5,CTF Finance Centre,Guangzhou,China,530,111,2015
6,Taipei 101,Taipei,Republic of China,509,101,2004
7,Shanghai World Financial Center,Shanghai,China,492,101,2008
8,International Commerce Centre The Ritz-Carlton...,Hong Kong,China,484,118,2010
9,Petronas Tower 1,Kuala Lumpur,Malaysia,452,88,1998


Get only the City and Building name   
`SELECT city, building FROM buildings LIMIT 3`

In [5]:
 df[["City", "Building"]][:3]

,City,Building
0,Dubai,Burj Khalifa
1,Shanghai,Shanghai Tower
2,Mecca,Abraj Al-Bait Clock Tower


What are the three oldest buildings in our data   
`SELECT city, building, built FROM buildings ORDER BY built LIMIT 3`


In [6]:
df.sort("Built")[["City", "Building", "Built"]][:3]

,City,Building,Built
74,New York City,Chrysler Building,1930
29,New York City,Empire State Building,1931
46,Chicago,John Hancock Center,1969


What the the three highest buildings?   
`SELECT city, building, height FROM buildings ORDER BY height desc LIMIT 3`

In [7]:
df.sort("Height", ascending=False)[["City", "Building", "Height"]][:3]


,City,Building,Height
0,Dubai,Burj Khalifa,828
1,Shanghai,Shanghai Tower,632
2,Mecca,Abraj Al-Bait Clock Tower,601


What are the three tallest buildings in Dubai?   

    SELECT city, building, height FROM buildings 
    WHERE city='Dubai' ORDER BY height LIMIT 3;
    

In [9]:
df[df["City"]=='Dubai'].sort("Height", ascending=False)[:3]


,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
15,Marina 101,Dubai,UAE,426,101,2015
19,Princess Tower,Dubai,UAE,414,101,2012


What are the five tallest buildings in Dubai or Chicago?

    SELECT * FROM buildings WHERE city='Dubai' 
    or city=='Chicago' ORDER BY height desc LIMIT 5;

In [10]:
df[(df["City"]=='Dubai') | (df["City"]=="Chicago")].sort("Height", ascending=False)[:5]

,Building,City,Country,Height,Floors,Built
0,Burj Khalifa,Dubai,UAE,828,163,2010
12,Willis Tower (formerly the Sears Tower),Chicago,United States,442,108,1974
15,Marina 101,Dubai,UAE,426,101,2015
17,Trump International Hotel and Towe,Chicago,United States,423,98,2009
19,Princess Tower,Dubai,UAE,414,101,2012


Among the buildings built in 20th century, which has the most floors?

    SELECT * FROM buildings WHERE built < 2000 
    ORDER BY floors DESC LIMIT 1;

In [11]:
df[df["Built"]<2000].sort("Floors", ascending=False)[:1]

,Building,City,Country,Height,Floors,Built
12,Willis Tower (formerly the Sears Tower),Chicago,United States,442,108,1974


Among the buildings built in 20th century, which is the tallest?

    SELECT * FROM buildings WHERE built < 2000
    ORDER BY height DESC LIMIT 1;

In [12]:
df[df["Built"]<2000].sort("Height", ascending=False)[:1]


,Building,City,Country,Height,Floors,Built
9,Petronas Tower 1,Kuala Lumpur,Malaysia,452,88,1998


How many different countries and cities do we have in our data?

    SELECT COUNT(DISTINCT(country)) FROM buildings;
    SELECT COUNT(DISTINCT(city)) FROM buildings;

In [13]:
len(df["Country"].unique())
len(df["City"].unique())

45

Which cities make an apperance in top 20 tallest building list?

    SELECT DISTINCT(city) from 
    (SELECT * FROM buildings ORDER BY height DESC LIMIT 20);
    

In [15]:
 df[:20].sort("Height", ascending=False)["City"].unique()

array(['Dubai', 'Shanghai', 'Mecca', 'Shenzhen', 'New York City',
       'Guangzhou', 'Taipei', 'Hong Kong', 'Kuala Lumpur', 'Nanjing',
       'Chicago'], dtype=object)